# Start code

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [3]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [4]:
train_dataset = KeypointsDataset("/kaggle/input/dataset/data/images","/kaggle/input/dataset/data/data_train.json")
val_dataset = KeypointsDataset("/kaggle/input/dataset/data/images","/kaggle/input/dataset/data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create model

In [5]:
model = models.resnet101(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 174MB/s]


In [6]:
model = model.to(device)

# Train model

In [7]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [8]:
epochs=50
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Step {i+1}/{len(train_loader)}, loss: {loss.item()}")

Epoch 1/50, Step 1/829, loss: 14598.2353515625
Epoch 1/50, Step 11/829, loss: 13355.0693359375
Epoch 1/50, Step 21/829, loss: 14926.6796875
Epoch 1/50, Step 31/829, loss: 13297.138671875
Epoch 1/50, Step 41/829, loss: 13309.8154296875
Epoch 1/50, Step 51/829, loss: 12984.458984375
Epoch 1/50, Step 61/829, loss: 12768.2314453125
Epoch 1/50, Step 71/829, loss: 12381.0830078125
Epoch 1/50, Step 81/829, loss: 12120.4306640625
Epoch 1/50, Step 91/829, loss: 10894.880859375
Epoch 1/50, Step 101/829, loss: 10757.314453125
Epoch 1/50, Step 111/829, loss: 10601.431640625
Epoch 1/50, Step 121/829, loss: 9640.603515625
Epoch 1/50, Step 131/829, loss: 10447.119140625
Epoch 1/50, Step 141/829, loss: 8920.927734375
Epoch 1/50, Step 151/829, loss: 9432.7314453125
Epoch 1/50, Step 161/829, loss: 8844.66015625
Epoch 1/50, Step 171/829, loss: 10344.453125
Epoch 1/50, Step 181/829, loss: 7711.51416015625
Epoch 1/50, Step 191/829, loss: 7880.08837890625
Epoch 1/50, Step 201/829, loss: 7024.4677734375
Epoc

In [9]:
torch.save(model.state_dict(), "keypoints_resnet_101.pth")